# Recruit Restaurant Visitor Forecasting
表題のコンペのコード<br>
処理時間およそ30min<br>
* 特徴量を作成(日付、店舗のピーク曜日、座標、カテゴリ)
* カテゴリ変数を変換(店舗IDと店舗カテゴリ)
* ラグ特徴量(7,14,21日前の平均と、直前の同じ曜日の来店者数)
<br><br>
【モデル学習方法】<br>
* 3/15-4/22までを予測させるためのモデル39個のうちの1個目
* 1日前までの情報を使って予測する

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
train = pd.read_csv("../input/recruit-restaurant-visitor-forecasting-data/air_visit_data.csv")
weatherData = pd.read_csv("../input/recruit-restaurant-visitor-forecasting-data/WeatherData.csv")
date_info = pd.read_csv("../input/recruit-restaurant-visitor-forecasting-data/date_info.csv")

In [ ]:
date_info = date_info.rename(columns={'calendar_date': "visit_date"})
date_info

In [ ]:
train = train.merge(date_info, on="visit_date",how="left")
# 曜日はあとで別の方法で加えるので、一旦削除
train = train.drop(columns="day_of_week")
train

In [ ]:
train["set"] = "train"
train

In [ ]:
train_x = train.drop(columns = "visitors")
train_y = pd.DataFrame()
train_y["visitors"] = train["visitors"]

In [ ]:
# train_yのvisitorsが200を超える場合は0にする
train_y.loc[train_y["visitors"] > 200, "visitors"] = 0
# visitorsでソートして表示
train_y.sort_values(by='visitors', ascending=False)

# 日付を切り分ける
毎月〇日は安い、とか、〇月はセール！とかありそうなので

In [ ]:
# 2017-04-18	
def back_year(txt):
    visit_date = txt
    txt_split = txt.rsplit("-",2)
    return txt_split[0]

def back_month(txt):
    visit_date = txt
    txt_split = txt.rsplit("-",2)
    #04月のような記述にならないように、int型で返す
    return int(txt_split[1])

def back_day(txt):
    visit_date = txt
    txt_split = txt.rsplit("-",2)
    return int(txt_split[2])

# year,month,dayの列を作成

In [ ]:
# year列を新規に作成する。visit_date列の値をback_year関数に入れて処理する
train_x.loc[:,"year"] = train_x["visit_date"].apply(back_year)
train_x.loc[:,"month"] = train_x["visit_date"].apply(back_month)
train_x.loc[:,"day"] = train_x["visit_date"].apply(back_day)
train_x

In [ ]:
import datetime
#月火水木金土日→[0,1,2,3,4,5,6]で返す
def back_day_of_the_week(txt):
    visit_date = txt
    txt_split = txt.rsplit("-",2)
    dt = datetime.datetime(int(txt_split[0]), int(txt_split[1]), int(txt_split[2]))
    return dt.weekday()

In [ ]:
train_x.loc[:,"day_of_the_week"] = train_x["visit_date"].apply(back_day_of_the_week)
# visit_dateは削除
train_x = train_x.drop(columns='visit_date')
train_x

# sample submissionを読み込み、testデータに使う

In [ ]:
test = pd.read_csv("../input/recruit-restaurant-visitor-forecasting-data/sample_submission.csv")
test["set"] = "test"
test_x = test.drop(columns="visitors")
test_y = pd.DataFrame()
test_y["visitors"] = test["visitors"]
# test_yの来店者数は全て-1で埋める(→ラグ特徴量計算でマイナスを平均に加えるのは不適切と考えやめた)
# test_y["visitors"] = -1
test_y

# test_xのidを切り分ける
test_x(sample_submit)のidが、店舗名と日付で別々になっているので、<br>
train_x(air_visit_data)と同じ形式(air_store_idとvisit_date)になるように<br>
切り分けたい

In [ ]:
def back_store_name(txt):
    id = txt
    #_区切りで右から1番目の文字を切り分ける
    txt_split = txt.rsplit("_",1)
    # 切り分けた文字列のうち、0番目を返す
    return txt_split[0]

def back_date_name(txt):
    id = txt
    txt_split = txt.rsplit("_",1)
    return txt_split[1]

# 新しく作った変数(air_store_idとvisit_date)と祝日フラグを、test_xに追加する

In [ ]:
#新しい列の名前 = 引数として関数に入れて処理する列の名前
test_x.loc[:,"air_store_id"] = test_x["id"].apply(back_store_name)
test_x.loc[:,"visit_date"] = test_x["id"].apply(back_date_name)
# 祝日フラグを追加
test_x = test_x.merge(date_info, on="visit_date",how="left")
# 曜日はあとで別の方法で加えるので、一旦削除
test_x = test_x.drop(columns="day_of_week")
train

# 日付情報をYear,Month,Dayに変更

In [ ]:
# year列を新規に作成する。visit_date列の値をback_year関数に入れて処理する
test_x.loc[:,"year"] = test_x["visit_date"].apply(back_year)
test_x.loc[:,"month"] = test_x["visit_date"].apply(back_month)
test_x.loc[:,"day"] = test_x["visit_date"].apply(back_day)
test_x

# 日付情報から曜日を取得

In [ ]:
test_x.loc[:,"day_of_the_week"] = test_x["visit_date"].apply(back_day_of_the_week)
test_x

# test_xのGWを休日(土曜)に変更
test_xに含まれるGW(5/3-5/5)の曜日を変更する<br>
条件によりday_of_the_weekを書き換える<br>
当初日曜日にしていたが、土曜日の方がave_visitorsが多いので土曜に変えた

In [ ]:
# test_dataの5/3-5/5までを日曜日に変更(日曜日=6)
test_x.loc[(test_x['month'] == 5) & (test_x['day'] == 3), 'day_of_the_week'] = 5
test_x.loc[(test_x['month'] == 5) & (test_x['day'] == 4), 'day_of_the_week'] = 5
test_x.loc[(test_x['month'] == 5) & (test_x['day'] == 5), 'day_of_the_week'] = 5
test_x

# idとvisit_date行は削除
idは不要。これで、test_xとtrain_xが同じ形式で取得できた<br>
visit_dateも、month,day,yearで代替しているので削除

In [ ]:
test_x = test_x.drop(columns='id')
test_x = test_x.drop(columns='visit_date')

# カテゴリ変数の変換
カテゴリ変数をLabelEncoderで実施するために、<br>
全てのカテゴリ変数(ここだと店舗名)を取得する<br>
# まず、air_store_idだけをもつDataFrameを定義する

In [ ]:
train_le = pd.DataFrame()
train_le["air_store_id"] = train_x.air_store_id
test_le = pd.DataFrame()
test_le["air_store_id"] = test_x.air_store_id
test_le

# 2つ(train_leとtest_le)を合体する

In [ ]:
merge_data_le = pd.merge(train_le,test_le, how="outer")
merge_data_le

# ラベルエンコーディングをかける
train_all_leというDataFrameにラベルエンコーディングをかける。<br>
air_store_idにラベルエンコーディングがかけられて数値に変換される<br>

In [ ]:
from sklearn.preprocessing import LabelEncoder #Library for LabelEncoding

for c in merge_data_le:
    le = LabelEncoder()
    le.fit(merge_data_le[c])
    train_le[c] = le.transform(train_le[c])
    test_le[c]= le.transform(test_le[c])

In [ ]:
# エンコード前後の対応確認用、本筋とは関係ない
train_x_enc = train_x.join(train_le, lsuffix='_enc')
test_x_enc = test_x.join(test_le, lsuffix='_enc')

# エンコードしたものを入れる
train_x = train_x.drop(columns = "air_store_id")
train_x = train_x.join(train_le)
test_x = test_x.drop(columns = "air_store_id")
test_x = test_x.join(test_le)

# yearのデータ型をint64に変更
xgboostで読める型に変更する

In [ ]:
train_x["year"] = train_x.year.astype("int64")
test_x["year"] = test_x.year.astype("int64")

# train_xとtrain_yを結合してtrain_allにする

In [ ]:
train_all = train_x.join(train_y)
train_all

# test_yとtest_xを結合してtest_allにする

In [ ]:
test_all = test_x.join(test_y)
test_all_wid = test_x.join(test.id)
test_all

# train_allとtest_allをマージしmerge_dataとする
これで、trainとtestが同じ形式で得られた。
以降は、train_allとtest_allをマージしてmerge_dataにして特徴量を追加し、<br>
最終的に切り分ける事にする

In [ ]:
merge_data = pd.concat([train_all,test_all])
merge_data

# train_allを使ってStore_idの入れ物をつくる

In [ ]:
unique_stores = train_all['air_store_id'].unique()
stores = pd.concat(
    [
        pd.DataFrame({
            'air_store_id': unique_stores,
            'day_of_the_week': [i] * len(unique_stores)
        }) for i in range(7)
    ],
    axis=0,
    ignore_index=True).reset_index(drop=True)

In [ ]:
stores

# 入れ物の中身を入れていく

In [ ]:
#sure it can be compressed...
tmp = train_all.groupby(
    ['air_store_id', 'day_of_the_week'],
    as_index=False)['visitors'].min().rename(columns={
        'visitors': 'min_visitors'
    })
stores = pd.merge(stores, tmp, how='left', on=['air_store_id', 'day_of_the_week'])

tmp = train_all.groupby(
    ['air_store_id', 'day_of_the_week'],
    as_index=False)['visitors'].mean().rename(columns={
        'visitors': 'mean_visitors'
    })
stores = pd.merge(stores, tmp, how='left', on=['air_store_id', 'day_of_the_week'])

tmp = train_all.groupby(
    ['air_store_id', 'day_of_the_week'],
    as_index=False)['visitors'].median().rename(columns={
        'visitors': 'median_visitors'
    })
stores = pd.merge(stores, tmp, how='left', on=['air_store_id', 'day_of_the_week'])

tmp = train_all.groupby(
    ['air_store_id', 'day_of_the_week'],
    as_index=False)['visitors'].max().rename(columns={
        'visitors': 'max_visitors'
    })
stores = pd.merge(stores, tmp, how='left', on=['air_store_id', 'day_of_the_week'])

tmp = train_all.groupby(
    ['air_store_id', 'day_of_the_week'],
    as_index=False)['visitors'].count().rename(columns={
        'visitors': 'count_observations'
    })
stores = pd.merge(stores, tmp, how='left', on=['air_store_id', 'day_of_the_week'])

stores

# merge_dataにmergeする

In [ ]:
merge_data = pd.merge(merge_data, stores, how='left', on=['air_store_id', 'day_of_the_week'])
merge_data

# 【important】validationデータの指定
3/15を予測する。<br>
それ以外の3/15-4-22の期間はmissデータとして、学習に利用しない

In [ ]:
# 3/15-4/22をmissデータとする
merge_data.loc[(merge_data['year'] == 2017) & (merge_data['month'] == 3) & (merge_data['day'] >= 12), 'set'] = 'miss'
merge_data.loc[(merge_data['year'] == 2017) & (merge_data['month'] == 4) & (merge_data['day'] >= 1) & (merge_data['day'] <= 22), 'set'] = 'miss'

# merge_dataの中で、予測したい日付を指定する
merge_data.loc[(merge_data['year'] == 2017) & (merge_data['month'] == 3) & (merge_data['day'] >= 18) & (merge_data['day'] <= 18),'set'] = 'va'

In [ ]:
va_miss_visitors = merge_data.loc[(merge_data.set == "va") | (merge_data.set == "miss"),"visitors"]
# set=vaに選んだもののvisitorsを取得しておく
va_visitors = merge_data.loc[(merge_data.set == "va"),"visitors"]

merge_data.loc[(merge_data.set == "va") | (merge_data.set == "miss"),"visitors"] = 0
merge_data.loc[(merge_data.set == "va") | (merge_data.set == "miss")]

# 店舗情報を利用する

In [ ]:
air_store_info = pd.read_csv("../input/recruit-restaurant-visitor-forecasting-data/air_store_info.csv")

store_le = pd.DataFrame()
store_le["air_store_id"] = air_store_info.air_store_id
genre_le = pd.DataFrame()
genre_le["air_genre_name"] = air_store_info.air_genre_name
area_le = pd.DataFrame()
area_le["air_area_name"] = air_store_info.air_area_name

area_le

# ラベルエンコーディングで店舗のジャンルを分ける
・store_id<br>
・ジャンル<br>
・エリア<br>
に対してそれぞれラベルエンコーディングする

In [ ]:
# store_idのラベルエンコーディング
for c in store_le:
    store_le[c] = le.transform(store_le[c])
store_le

# ジャンルのラベルエンコーディング
for c in genre_le:
    le_g = LabelEncoder()
    le_g.fit(genre_le[c])
    genre_le[c] = le_g.transform(genre_le[c])
genre_le

for c in area_le:
    le_a = LabelEncoder()
    le_a.fit(area_le[c])
    area_le[c] = le_a.transform(area_le[c])
area_le

# ラベルエンコーディングした特徴量を結合
これで、店舗IDごとに座標、エリア、ジャンルが入ったDataframe「air_store_info」を得られる

In [ ]:
air_store_info = air_store_info.drop(columns = "air_store_id")
air_store_info = air_store_info.join(store_le)
air_store_info = air_store_info.drop(columns = "air_genre_name")
air_store_info = air_store_info.join(genre_le)
air_store_info = air_store_info.drop(columns = "air_area_name")
air_store_info = air_store_info.join(area_le)
# 最も来店者の多い曜日情報と結合するためにソートしindexをつけなおす
air_store_info = air_store_info.sort_values(by='air_store_id')
# indexをつけなおす
air_store_info = air_store_info.reset_index(drop=True)
air_store_info

In [ ]:
store_visitors = train_all.groupby(['air_store_id']).visitors.agg([sum,len,max,min])
store_visitors["ave"] = store_visitors["sum"] // store_visitors["len"]
# 来店者数の平均でソート
store_visitors.sort_values(by='ave', ascending=False)

# merge_dataに店舗情報を追加
vlookupみたいな操作をする。参照して追加するみたいな<br>
merge_dataには、trainもtestも、xもyも全部含まれる<br>
air_store_infoの店舗IDごとの情報を、各データに追加する

In [ ]:
merge_data = merge_data.merge(air_store_info, on="air_store_id",how="left")
merge_data

# ラグ特徴量の追加
 7,14,21日周期のラグ特徴量を算出してlag_allに持たせる<br>
 testデータでは当日の情報を使ってラグ特徴量を作る事ができないので<br>
 1日シフトさせて、前日からの1週間の平均を取得する

In [ ]:
%%time
store_all_14_max = pd.DataFrame()
store_all_14 = pd.DataFrame()
store_all_21_max = pd.DataFrame()
store_all_21 = pd.DataFrame()
store_all_28_max = pd.DataFrame()
store_all_28 = pd.DataFrame()
store_all_35_max = pd.DataFrame()
store_all_35 = pd.DataFrame()
store_all_42_max = pd.DataFrame()
store_all_42 = pd.DataFrame()

for c in range(0,829,1):
    
    # cで指定した店舗idを持つDataFrameを拾ってくる(店舗別にラグ特徴量を得るために)
    store = merge_data.loc[merge_data.air_store_id == c]
    # 14周期前のlagを取得
    x_lag = pd.DataFrame()
    x_lag["lag14_max"] = store["visitors"].shift(8).rolling(window=7).max()
    store = store.join(x_lag)
    # 欠損値は、その店舗の平均来店者数で埋める
    ave = store_visitors['max'][c]
    store = store.fillna(ave)
    store_all_14_max = pd.concat([store_all_14_max, store])
    
    # cで指定した店舗idを持つDataFrameを拾ってくる(店舗別にラグ特徴量を得るために)
    store = merge_data.loc[merge_data.air_store_id == c]
    # 14周期前のlagを取得
    x_lag = pd.DataFrame()
    x_lag["lag14"] = store["visitors"].shift(8).rolling(window=7).mean()
    store = store.join(x_lag)
    # 欠損値は、その店舗の平均来店者数で埋める
    ave = store_visitors['ave'][c]
    store = store.fillna(ave)
    store_all_14 = pd.concat([store_all_14, store])
    
    # cで指定した店舗idを持つDataFrameを拾ってくる(店舗別にラグ特徴量を得るために)
    store = merge_data.loc[merge_data.air_store_id == c]
    # 14周期前のlagを取得
    x_lag = pd.DataFrame()
    x_lag["lag21_max"] = store["visitors"].shift(15).rolling(window=7).max()
    store = store.join(x_lag)
    # 欠損値は、その店舗の平均来店者数で埋める
    ave = store_visitors['max'][c]
    store = store.fillna(ave)
    store_all_21_max = pd.concat([store_all_21_max, store])
    
    # cで指定した店舗idを持つDataFrameを拾ってくる(店舗別にラグ特徴量を得るために)
    store = merge_data.loc[merge_data.air_store_id == c]
    # 21周期前のlagを取得
    x_lag = pd.DataFrame()
    x_lag["lag21"] = store["visitors"].shift(15).rolling(window=7).mean()
    store = store.join(x_lag)
    # 欠損値は、その店舗の平均来店者数で埋める
    ave = store_visitors['ave'][c]
    store = store.fillna(ave)
    store_all_21 = pd.concat([store_all_21, store])
    
    # cで指定した店舗idを持つDataFrameを拾ってくる(店舗別にラグ特徴量を得るために)
    store = merge_data.loc[merge_data.air_store_id == c]
    # 14周期前のlagを取得
    x_lag = pd.DataFrame()
    x_lag["lag28_max"] = store["visitors"].shift(22).rolling(window=7).max()
    store = store.join(x_lag)
    # 欠損値は、その店舗の平均来店者数で埋める
    ave = store_visitors['max'][c]
    store = store.fillna(ave)
    store_all_28_max = pd.concat([store_all_28_max, store])
    
    # cで指定した店舗idを持つDataFrameを拾ってくる(店舗別にラグ特徴量を得るために)
    store = merge_data.loc[merge_data.air_store_id == c]
    # 21周期前のlagを取得
    x_lag = pd.DataFrame()
    x_lag["lag28"] = store["visitors"].shift(22).rolling(window=7).mean()
    store = store.join(x_lag)
    # 欠損値は、その店舗の平均来店者数で埋める
    ave = store_visitors['ave'][c]
    store = store.fillna(ave)
    store_all_28 = pd.concat([store_all_28, store])
    
        # cで指定した店舗idを持つDataFrameを拾ってくる(店舗別にラグ特徴量を得るために)
    store = merge_data.loc[merge_data.air_store_id == c]
    # 14周期前のlagを取得
    x_lag = pd.DataFrame()
    x_lag["lag35_max"] = store["visitors"].shift(29).rolling(window=7).max()
    store = store.join(x_lag)
    # 欠損値は、その店舗の平均来店者数で埋める
    ave = store_visitors['max'][c]
    store = store.fillna(ave)
    store_all_35_max = pd.concat([store_all_35_max, store])
        
    # cで指定した店舗idを持つDataFrameを拾ってくる(店舗別にラグ特徴量を得るために)
    store = merge_data.loc[merge_data.air_store_id == c]
    # 21周期前のlagを取得
    x_lag = pd.DataFrame()
    x_lag["lag35"] = store["visitors"].shift(29).rolling(window=7).mean()
    store = store.join(x_lag)
    # 欠損値は、その店舗の平均来店者数で埋める
    ave = store_visitors['ave'][c]
    store = store.fillna(ave)
    store_all_35 = pd.concat([store_all_35, store])
    
        # cで指定した店舗idを持つDataFrameを拾ってくる(店舗別にラグ特徴量を得るために)
    store = merge_data.loc[merge_data.air_store_id == c]
    # 14周期前のlagを取得
    x_lag = pd.DataFrame()
    x_lag["lag42_max"] = store["visitors"].shift(36).rolling(window=7).max()
    store = store.join(x_lag)
    # 欠損値は、その店舗の平均来店者数で埋める
    ave = store_visitors['max'][c]
    store = store.fillna(ave)
    store_all_42_max = pd.concat([store_all_42_max, store])
    
    # cで指定した店舗idを持つDataFrameを拾ってくる(店舗別にラグ特徴量を得るために)
    store = merge_data.loc[merge_data.air_store_id == c]
    # 21周期前のlagを取得
    x_lag = pd.DataFrame()
    x_lag["lag42"] = store["visitors"].shift(36).rolling(window=7).mean()
    store = store.join(x_lag)
    # 欠損値は、その店舗の平均来店者数で埋める
    ave = store_visitors['ave'][c]
    store = store.fillna(ave)
    store_all_42= pd.concat([store_all_42, store])

In [ ]:
# idでソートしなおす
store_all_14 = store_all_14.sort_index()
store_all_21 = store_all_21.sort_index()
store_all_28 = store_all_28.sort_index()
store_all_35 = store_all_35.sort_index()
store_all_42 = store_all_42.sort_index()
store_all_14_max = store_all_14_max.sort_index()
store_all_21_max = store_all_21_max.sort_index()
store_all_28_max = store_all_28_max.sort_index()
store_all_35_max = store_all_35_max.sort_index()
store_all_42_max = store_all_42_max.sort_index()
# ラグ特徴量を結合してlag_all
lag_all = pd.DataFrame()
lag_all["lag14"] = store_all_14.lag14
lag_all["lag21"] = store_all_21.lag21
lag_all["lag28"] = store_all_28.lag28
lag_all["lag35"] = store_all_35.lag35
lag_all["lag42"] = store_all_42.lag42
lag_all["lag14_max"] = store_all_14_max.lag14_max
lag_all["lag21_max"] = store_all_21_max.lag21_max
lag_all["lag28_max"] = store_all_28_max.lag28_max
lag_all["lag35_max"] = store_all_35_max.lag35_max
lag_all["lag42_max"] = store_all_42_max.lag42_max
lag_all.head(30)

# ラグ特徴量をmerge_dataに追加

In [ ]:
merge_data = merge_data.join(lag_all)
merge_data

In [ ]:
# リークする危険があるのでか必ずtodayをdropして消す
# merge_data = merge_data.drop(columns='today')
# merge_data = merge_data.drop(columns='day_of_the_week_last')

In [ ]:
for col in merge_data:
    print(col)

# 全期間のデータをmerge_data_allとして残す

In [ ]:
#期間短くする前のmerge_dataは、merge_data_all(全期間のデータ)として残す
merge_data_all = merge_data
merge_data_all

In [ ]:
# merge_data_allへはここで修復できる
merge_data = merge_data_all
merge_data

In [ ]:
merge_data.loc[merge_data.month == 3].tail(20)

In [ ]:
merge_data.loc[merge_data.set == "va"]

In [ ]:
merge_data.loc[merge_data.set == "miss"]

# 学習データを減らして学習時間を削減する

In [ ]:
merge_data = merge_data.loc[(merge_data['year'] == 2017)]
merge_data

# 以上で、特徴量エンジニアリングは終了
パラメータを調整し、予測できるようにする

* vaデータとして使う日付を選択する
* データをtrain_fitとtest_fitに分割する
* xgboostのパラメータを指定 
* xgboostで学習させる
* vaデータでRMSLEを計算する


# model1

# テストデータの範囲を指定
パラメータが定まり、4/22までのデータを全てtrainとして使う設定にした場合に行う

In [ ]:
merge_data.loc[(merge_data['year'] == 2017) & (merge_data['month'] == 4) & (merge_data['day'] >= 24), 'set'] = 'test'
merge_data.loc[(merge_data['year'] == 2017) & (merge_data['month'] == 5) & (merge_data['day'] >= 1), 'set'] = 'test'

# testのidを残したい
test_all_wid.loc[(test_all_wid['year'] == 2017) & (test_all_wid['month'] == 4) & (test_all_wid['day'] >= 30), 'set'] = 'test'
test_all_wid.loc[(test_all_wid['year'] == 2017) & (test_all_wid['month'] == 5) & (test_all_wid['day'] >= 1), 'set'] = 'test'

In [ ]:
# train,test,vaに分ける
train_fit = pd.DataFrame()
train_fit = merge_data.loc[merge_data.set == "train"]
train_fit_y = pd.DataFrame()
train_fit_y["visitors"] = train_fit.visitors
train_fit_x = train_fit.drop(columns="visitors")
train_fit_x = train_fit_x.drop(columns="set")

test_fit = pd.DataFrame()
test_fit = merge_data.loc[merge_data.set == "test"]
test_fit_x = test_fit.drop(columns="visitors")
test_fit_x = test_fit_x.drop(columns="set")

test_id = pd.DataFrame()
test_id = test_all_wid.loc[test_all_wid.set == "test"]

va_fit = pd.DataFrame()
va_fit = merge_data.loc[merge_data.set == "va"]
va_fit_y = pd.DataFrame()
va_fit_y["visitors"] = va_fit.visitors
va_fit_x = va_fit.drop(columns="visitors")
va_fit_x = va_fit_x.drop(columns="set")



In [ ]:
from xgboost import XGBClassifier as XGB
from sklearn.metrics import mean_squared_log_error

Model = XGB(n_estimators=10, random_state=71,max_depth=10,colsample_bylevel=0.4,
           gamma=0.0,alpha=0.0, min_child_weight=4,subsample=0.8,colsample_bytree=0.95
           )

# 特徴量を落とす

In [ ]:
# 落とす特徴量を選ぶ
drop_columns = ["lag14","month","day","lag21","lag28","lag35","lag42","lag14_max","lag21_max","lag28_max","lag35_max","lag42_max"]
train_fit_x = train_fit_x.drop(columns=drop_columns)
test_fit_x = test_fit_x.drop(columns=drop_columns)
va_fit_x = va_fit_x.drop(columns=drop_columns)

In [ ]:
va_fit_x

In [ ]:
train_fit_x.nunique()

In [ ]:
%%time
Model.fit(train_fit_x, train_fit_y)

In [ ]:
# モデルで予測する
pred = Model.predict(va_fit_x)

# 予測値からRMSLEを算出する
score = np.sqrt(mean_squared_log_error(va_visitors, pred))
print("---------------RMSLE-score----------------")
score

In [ ]:
columns = ["pred_visitors"]
pred_d = pd.DataFrame(data=pred, columns = columns)
columns = ["visitors"]
va_fit_d = pd.DataFrame(data=va_visitors, columns = columns)
va_fit_d = va_fit_d.reset_index(drop=True)

sns.scatterplot(x=va_fit_d['visitors'], y=pred_d['pred_visitors'])
sns.scatterplot(x=va_fit_d['visitors'], y=va_fit_d['visitors']) #perfect fitting line

# 以下、何が予測できていないのか確認

In [ ]:
va_fit_d["delta"] = va_fit_d["visitors"] - pred_d["pred_visitors"]
va_fit_d["RMSLE"] = np.sqrt(np.log10(va_fit_d["visitors"]+1) - np.log10(pred_d["pred_visitors"]+1)**2)
va_fit_x_reindex = va_fit_x.reset_index(drop=True)
va_fit_d = va_fit_d.join(va_fit_x_reindex)
va_fit_d = va_fit_d.join(pred_d)
va_fit_d["delta_abs"] = abs(va_fit_d["delta"])
va_fit_d = va_fit_d.sort_values(by='RMSLE', ascending=False)


In [ ]:
# va_fit_d_watch= va_fit_d[["RMSLE","delta_abs","visitors","pred_visitors","year","month","day","air_store_id","day_of_the_week"]]
# va_fit_d_watch.head(50)

In [ ]:
sns.scatterplot(x=va_fit_d_watch['delta_abs'], y=va_fit_d_watch['RMSLE'])

In [ ]:
va_fit_d_bad = va_fit_d.sort_values(by='RMSLE', ascending=False).head(100)
va_fit_d_watch_bad = va_fit_d_watch.sort_values(by='delta_abs', ascending=False).head(100)

In [ ]:
va_fit_d_watch_bad.sort_values(by='day_of_the_week', ascending=False).head(50)

In [ ]:
sns.distplot(
    va_fit_d_watch_bad['day_of_the_week'], bins=100, color='#123456', label='data',
    kde=False,
    rug=False
)

# testデータの予測

In [ ]:
# モデルで予測する
pred = Model.predict(test_fit_x)
pred

In [ ]:
columns_ck = [["visitors"]]
submission_ck = pd.DataFrame(data=pred, columns = columns_ck)
submission_ck

In [ ]:
test_id = test_id.reset_index(drop=True)
test_id

In [ ]:
submission = pd.DataFrame()
submission["id"] = test_id.id
submission["visitors"] = submission_ck.visitors
submission

In [ ]:
submission.to_csv("submission_week1.csv", index=False)